## Parsing Mod dataset to YOLOv5 extracted feature

In [1]:
# Utilities

import numpy as np
import torch
import pandas as pd
import base64
from PIL import Image
import io

def base64toPilImage(im_b64):
    im_bytes = base64.b64decode(im_b64)   # im_bytes is a binary image
    im_file = io.BytesIO(im_bytes)  # convert image to file-like object
    return Image.open(im_file).convert("RGB")   # img is now PIL Image object

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\danil/.cache\torch\hub\ultralytics_yolov5_master
Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
YOLOv5  2021-6-21 torch 1.8.0 CUDA:0 (NVIDIA GeForce RTX 2060, 6144.0MB)



### Import Dataset

In [5]:
# Filtered fields read from dataset csv file
fields = [
    'Distance',
    'PedTopRightX', 
    'PedTopRightY',
    'PedBottomLeftX', 
    'PedBottomLeftY',
    'B64File'
]

df = pd.read_csv (r'./Dataset.csv', usecols=fields)
print(df.shape)

(10022, 6)


### Parsing Dataset

In [6]:
yoloNumpy = []
# Images
for index in df.index:   
    img = base64toPilImage(df.B64File[index])
    # Inference
    results = model(img)
    obj_recognized = results.xyxy[0].cpu().numpy()
    for obj in obj_recognized:
        if obj[5] == 0:
            yoloNumpy.append([df.Distance[index], obj[0], obj[1], obj[2], obj[3], df.B64File[index]])          

yoloDataframe = pd.DataFrame(yoloNumpy, columns = [ 'Distance', 'Xmin', 'Ymin', 'Xmax', 'Ymax', 'B64File'])
yoloDataframe.to_csv('yoloDataset.csv', index=False)
print(yoloDataframe.shape)

(4382, 6)
